In [1]:
import matplotlib.pyplot as plt
import numpy as np
from open_ephys.analysis import Session
import os
import re
from scipy.signal import butter, filtfilt, lfilter
from collections import defaultdict
import scipy.stats as stats


# ==== Select Session ====
w = 1  # recordnode index 1 = Record Node 111 RAW is Offline Filtered
u = 0       # recordnode index 0 = Record Node 106 is Online Filtered

In [2]:
def scan_experiment_structure(session_dir, record_node_name="Record Node 106"):
    node_path = os.path.join(session_dir, record_node_name)
    
    if not os.path.exists(node_path):
        print(f"Record Node directory not found: {node_path}")
        return {}
    
    experiment_info = defaultdict(list)
    for item in sorted(os.listdir(node_path)):
        exp_path = os.path.join(node_path, item)
        if os.path.isdir(exp_path) and item.startswith("experiment"):
            recordings = [r for r in sorted(os.listdir(exp_path)) 
                          if os.path.isdir(os.path.join(exp_path, r)) and r.startswith("recording")]
            experiment_info[item] = recordings
    return experiment_info

In [3]:
# Load Session
directory = 'dEMG_Pilot-0012025-10-31_13-23-53_001'

session = Session(directory)
print('Pass Initial Test, Moving on to Loading Session...\n')

# === Scan structure ===
record_node_name = "Record Node 106" if w == 0 else "Record Node 111"
structure = scan_experiment_structure(directory, record_node_name)

print(session)

Pass Initial Test, Moving on to Loading Session...


Open Ephys Recording Session Object
Directory: dEMG_Pilot-0012025-10-31_13-23-53_001

<object>.recordnodes:
  Index 0: Record Node 106 (binary format)
  Index 1: Record Node 111 (binary format)



In [4]:
# === Flatten experiment-recording pairs to loop ===
flat_recordings = []
for exp_name, rec_list in structure.items():
    for rec in rec_list:
        flat_recordings.append((exp_name, rec))

In [5]:
x = 0
exp = "experiment1"

y = f"recording{x+1}"  # sequential label used in some parts of your code
yy = y
#y = f"recording{x}"  # sequential label used in some parts of your code
v = f"Record Node 106" if w == 0 else f"Record Node 111"
vv = f"Record Node 111" if u == 0 else f"Record Node 106"
i = 1

try:
    recording = session.recordnodes[w].recordings[x]
    #recording_offline = session.recordnodes[w].recordings[x]
except IndexError:
    print(f"No recording {x} found in {exp}/{yy}")
    

print(f"Processing: x={x}, y={y}, exp={exp}, yy={yy}")

print(recording)

Processing: x=0, y=recording1, exp=experiment1, yy=recording1
Open Ephys GUI Recording
ID: 0x16a9c8a83e0
Format: Binary
Directory: dEMG_Pilot-0012025-10-31_13-23-53_001\Record Node 111\experiment1\recording1
Experiment Index: 0
Recording Index: 0


In [7]:
# Extract metadata and global timestamps
metadata = recording.continuous[0].metadata
print(metadata, '\n')
metadata['sample_rate']
#channel_names = metadata['channel_names']
#print("Channels:", channel_names, '\n')



ContinuousMetadata(source_node_id=100, source_node_name='Acquisition Board', stream_name='Rhythm Data', sample_rate=25000.0, num_channels=5, channel_names=['CH1', 'CH5', 'CH7', 'CH11', 'ADC1'], bit_volts=[0.1949999928474426, 0.1949999928474426, 0.1949999928474426, 0.1949999928474426, 0.000152587890625]) 



TypeError: 'ContinuousMetadata' object is not subscriptable

In [21]:
recording.continuous

ContinuousDict({'Rhythm Data': <open_ephys.analysis.formats.BinaryRecording.BinaryContinuous object at 0x0000023F5CC27ED0>})

Data shape: (7619092, 5)
